In [1]:
!pip install streamlit
!npm install localtunnel
!pip install pycountry

npm WARN saveError ENOENT: no such file or directory, open '/content/package.json'
npm WARN enoent ENOENT: no such file or directory, open '/content/package.json'
npm WARN content No description
npm WARN content No repository field.
npm WARN content No README data
npm WARN content No license field.

+ localtunnel@2.0.2
updated 1 package and audited 36 packages in 0.82s

3 packages are looking for funding
  run `npm fund` for details

found 2 moderate severity vulnerabilities
  run `npm audit fix` to fix them, or `npm audit` for details


In [2]:
%%writefile app2.py

import streamlit as st
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
import pycountry

# Función para convertir ISO2 a ISO3
def iso2_to_iso3(iso2):
    try:
        return pycountry.countries.get(alpha_2=iso2).alpha_3
    except:
        return None

# Cargar los datos
df = pd.read_csv('ds_salaries.csv')

# Configurar el estado de sesión
if 'page' not in st.session_state:
    st.session_state.page = 0

# Definir las funciones para los gráficos
def average_salary_by_job_title():
    st.header("Average Salary by Job Title")
    df_fil = df[['job_title', 'salary_in_usd']]
    job_title_counts = df_fil['job_title'].value_counts()
    job_titles_filtered = job_title_counts[job_title_counts >= 10].index
    df_filtered = df[df['job_title'].isin(job_titles_filtered)]
    average_salary_by_job_title = df_filtered.groupby('job_title')['salary_in_usd'].mean().sort_values()

    plt.figure(figsize=(8, 10))
    sns.barplot(x=average_salary_by_job_title.values, y=average_salary_by_job_title.index, palette='viridis')
    plt.title('Average Salary by Job Title')
    plt.xlabel('Average Salary (USD)')
    plt.ylabel('Job Title')
    st.pyplot(plt)

def salary_distribution_by_experience_level():
    st.header("Salary Distribution by Experience Level")
    df_experience = df[['experience_level', 'salary_in_usd']]
    experience_order = ['EN', 'MI', 'SE', 'EX']
    df_experience['experience_level'] = pd.Categorical(df_experience['experience_level'], categories=experience_order, ordered=True)

    plt.figure(figsize=(12, 8))
    for level in experience_order:
        subset = df_experience[df_experience['experience_level'] == level]
        sns.kdeplot(subset['salary_in_usd'], label=level, fill=True)

    plt.title('Salary Distribution by Experience Level')
    plt.xlabel('Salary (USD)')
    plt.ylabel('Density')
    plt.legend(title='Experience Level')
    st.pyplot(plt)

def average_salary_by_company_size_and_experience_level():
    st.header("Average Salary by Company Size and Experience Level")
    df_company = df[['company_size', 'experience_level', 'salary_in_usd']]
    salary_by_company_and_experience = df_company.groupby(['company_size', 'experience_level'])['salary_in_usd'].mean().unstack()

    salary_by_company_and_experience.plot(kind='bar', stacked=True, figsize=(12, 8), colormap='viridis')
    plt.title('Average Salary by Company Size and Experience Level')
    plt.xlabel('Company Size')
    plt.ylabel('Average Salary (USD)')
    plt.legend(title='Experience Level')
    st.pyplot(plt)

def average_salary_by_remote_work_ratio():
    st.header("Average Salary by Remote Work Ratio")
    df_remote = df[['remote_ratio', 'salary_in_usd']]
    average_salary_by_remote_ratio = df_remote.groupby('remote_ratio')['salary_in_usd'].mean().sort_values()

    plt.figure(figsize=(12, 8))
    sns.barplot(x=average_salary_by_remote_ratio.index, y=average_salary_by_remote_ratio.values, palette='viridis')
    plt.title('Average Salary by Remote Work Ratio')
    plt.xlabel('Remote Work Ratio (%)')
    plt.ylabel('Average Salary (USD)')
    st.pyplot(plt)

def average_salary_by_country():
    st.header("Average Salary by Country")
    df_country = df[['employee_residence', 'salary_in_usd']]
    average_salary_by_country = df_country.groupby('employee_residence')['salary_in_usd'].median().reset_index()
    average_salary_by_country['iso_alpha'] = average_salary_by_country['employee_residence'].apply(iso2_to_iso3)

    fig = px.choropleth(
        average_salary_by_country,
        locations="iso_alpha",
        color="salary_in_usd",
        hover_name="employee_residence",
        color_continuous_scale=px.colors.sequential.Plasma,
        title="Average Salary by Country"
    )

    fig.update_layout(
        title='Average Salary by Country',
        geo=dict(showframe=False, showcoastlines=False, projection_type='equirectangular')
    )

    st.plotly_chart(fig)

def median_salaries_by_job_title_and_experience_level():
    st.header("Median Salaries by Job Title and Experience Level")
    salary_stats = df.groupby(['job_title', 'experience_level']).agg(
        median_salary=('salary_in_usd', 'median'),
        mean_salary=('salary_in_usd', 'mean'),
        min_salary=('salary_in_usd', 'min'),
        max_salary=('salary_in_usd', 'max'),
        count=('salary_in_usd', 'count'),
    ).reset_index()

    salary_stats = salary_stats[salary_stats['count'] > 10]
    experience_mapping = {'EN': 1, 'MI': 2, 'SE': 3, 'EX': 4}
    salary_stats['experience'] = salary_stats['experience_level'].map(experience_mapping)
    salary_stats = salary_stats.sort_values('experience')

    fig = go.Figure()

    for level in salary_stats['experience_level'].unique():
        subset = salary_stats[salary_stats['experience_level'] == level]
        fig.add_trace(go.Scatter(
            x=subset['experience_level'],
            y=subset['median_salary'],
            mode='markers',
            marker=dict(size=10),
            name=level,
            text=subset.apply(
                lambda row: f"Role: {row['job_title']}<br>Median: ${row['median_salary']:.1f}<br>Mean: ${row['mean_salary']:.1f}<br>Min: ${row['min_salary']:.1f}<br>Max: ${row['max_salary']:.1f}",
                axis=1
            ),
            hoverinfo='text'
        ))

    fig.update_layout(
        title='Median Salaries by Job Title and Experience Level',
        xaxis_title='Experience Level',
        yaxis_title='Median Salary (USD)',
        hovermode='closest'
    )

    st.plotly_chart(fig)

# Lista de gráficos
visualizations = [
    average_salary_by_job_title,
    salary_distribution_by_experience_level,
    average_salary_by_company_size_and_experience_level,
    average_salary_by_remote_work_ratio,
    average_salary_by_country,
    median_salaries_by_job_title_and_experience_level
]

# Mostrar gráfico actual
visualizations[st.session_state.page]()

# Botones de navegación
col1, col2, col3 = st.columns([1, 1, 1])

if col1.button('Previous'):
    if st.session_state.page > 0:
        st.session_state.page -= 1

if col3.button('Next'):
    if st.session_state.page < len(visualizations) - 1:
        st.session_state.page += 1

st.write(f"Visualization {st.session_state.page + 1} of {len(visualizations)}")


Overwriting app2.py


In [ ]:
!streamlit run app2.py &>/content/logs.txt & npx localtunnel --port 8501 & curl ipv4.icanhazip.com